In [2]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

/Users/tu/SourceCode/dirty_data_resolver/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

Extracting file to /Users/tu/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [4]:
# A DGL Dataset object may contain one or multiple graphs. The Cora dataset used in this tutorial only consists of one single graph.
g = dataset[0]

In [6]:
print("Node features")
print(g.ndata.keys())
print("Edge features")
print(g.edata)

Node features
dict_keys(['train_mask', 'val_mask', 'test_mask', 'label', 'feat'])
Edge features
{}


In [13]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [14]:

from dgl.data import DGLDataset

def train(g:DGLDataset, model: GCN):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )



In [15]:
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.072 (best 0.072), test acc: 0.095 (best 0.095)
In epoch 5, loss: 1.894, val acc: 0.522 (best 0.596), test acc: 0.520 (best 0.601)
In epoch 10, loss: 1.813, val acc: 0.598 (best 0.598), test acc: 0.610 (best 0.610)
In epoch 15, loss: 1.706, val acc: 0.700 (best 0.700), test acc: 0.709 (best 0.709)
In epoch 20, loss: 1.574, val acc: 0.696 (best 0.708), test acc: 0.702 (best 0.713)
In epoch 25, loss: 1.421, val acc: 0.694 (best 0.708), test acc: 0.697 (best 0.713)
In epoch 30, loss: 1.253, val acc: 0.700 (best 0.708), test acc: 0.712 (best 0.713)
In epoch 35, loss: 1.079, val acc: 0.712 (best 0.712), test acc: 0.723 (best 0.723)
In epoch 40, loss: 0.909, val acc: 0.726 (best 0.726), test acc: 0.732 (best 0.731)
In epoch 45, loss: 0.750, val acc: 0.742 (best 0.742), test acc: 0.741 (best 0.738)
In epoch 50, loss: 0.610, val acc: 0.750 (best 0.750), test acc: 0.753 (best 0.753)
In epoch 55, loss: 0.492, val acc: 0.770 (best 0.770), test acc: 0.766 (best 0

In [19]:
g.ndata["feat"][0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])